In [244]:
import pypsa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
from pypsa.plot import add_legend_lines, add_legend_patches, add_legend_semicircles
import yaml



In [245]:
fn = 'resources/DK_test/networks/base_s_2__12h_2050.nc'
n = pypsa.Network(fn)

INFO:pypsa.network.io:New version 1.0.3 available! (Current: 0.35.2)
INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, carriers, generators, global_constraints, links, loads, stores


In [246]:
cf_onshorewind=n.generators_t['p_max_pu'].loc[:, n.generators_t['p_max_pu'].columns.astype(str).str.contains('onwind')].sum()
cf_solar=n.generators_t['p_max_pu'].loc[:, n.generators_t['p_max_pu'].columns.astype(str).str.contains('solar')].sum().sort_values(ascending=False)



In [ ]:
n.add("Bus",
      name="DK0 cluster",
      v_nom=n.buses.at["DK0 0", "v_nom"],
      x=n.buses.at["DK0 0", "x"],
      y=n.buses.at["DK0 0", "y"],
      unit=n.buses.at["DK0 0", "unit"],
      location=n.buses.at["DK0 0", "location"],
      country="DK",
      carrier='AC',
      control=n.buses.at["DK0 0", "control"],
      substation_lv=n.buses.at["DK0 0", "substation_lv"],
      substation_off=n.buses.at["DK0 0", "substation_off"],
      )

n.add("Bus",
      name="DK0 cluster H2",
      country="DK",
      x=n.buses.at["DK0 0", "x"],
      y=n.buses.at["DK0 0", "y"],
      location=n.buses.at["DK0 0", "location"],
      unit=n.buses.at["DK0 0 H2", "unit"],
      carrier='H2')



Index(['DK0 cluster methanol'], dtype='object')

In [248]:
n.add(
        "Link",
        name="DK0 cluster H2 Electrolysis",
        bus1="DK0 cluster H2",
        bus0="DK0 cluster",
        p_nom_extendable=True,
        carrier="H2 Electrolysis",
        efficiency=n.links.at["DK0 0 H2 Electrolysis", "efficiency"],
        capital_cost=float(n.links.at["DK0 0 H2 Electrolysis", "capital_cost"])*0.8,
        marginal_cost=float(n.links.at["DK0 0 H2 Electrolysis", "marginal_cost"])*0.8,
        lifetime=n.links.at["DK0 0 H2 Electrolysis", "lifetime"],
        reversed=False,
        overwrite=True,
    )

n.add(
        "Link",
        name="DK0 cluster methanolization",
        bus0="DK0 cluster H2",
        bus1="EU methanol",
        bus2="DK0 cluster",
        bus3="DK0 0 co2 stored",
        bus4=n.links.at["DK0 0 methanolisation", "bus4"],
        carrier="methanolisation",
        p_nom_extendable=True,
        p_min_pu=n.links.at["DK0 0 methanolisation", "p_min_pu"],
        capital_cost=n.links.at["DK0 0 methanolisation", "capital_cost"]*0.8,
        marginal_cost=n.links.at["DK0 0 methanolisation", "marginal_cost"]*0.8,
        lifetime=n.links.at["DK0 0 methanolisation", "lifetime"],
        efficiency=n.links.at["DK0 0 methanolisation", "efficiency"],
        efficiency2=n.links.at["DK0 0 methanolisation", "efficiency2"],
        efficiency3=n.links.at["DK0 0 methanolisation", "efficiency3"],
        efficiency4=n.links.at["DK0 0 methanolisation", "efficiency4"],
        reversed=False,
        overwrite=True,
    )

# n.remove(
#         "Link",
#         name="DK0 cluster electricity",
#     )

# n.add(
#         "Link",
#         name="DK0 cluster electricity",
#         bus0="DK0 cluster",
#         bus1="DK0 0",
#         carrier="AC",   
#         p_nom_extendable=True,
#         efficiency=1.0,
#         capital_cost=0.0,
#         marginal_cost=0.0,
#     )

n.add(
        "Generator",
        name="DK0 0 cluster 4 onwind",
        bus="DK0 cluster",
        carrier="onwind",
        p_nom_max=n.generators.at["DK0 0 4 onwind", "p_nom_max"],
        p_max_pu=n.generators.at["DK0 0 4 onwind", "p_max_pu"],
        marginal_cost=n.generators.at["DK0 0 4 onwind", "marginal_cost"],
        capital_cost=n.generators.at["DK0 0 4 onwind", "capital_cost"],
        efficiency=n.generators.at["DK0 0 4 onwind", "efficiency"],
        p_nom_extendable=True,)

Index(['DK0 0 cluster 4 onwind'], dtype='object')

In [249]:
n.links.loc[n.links["carrier"]=='methanolisation']

,bus0,bus1,type,carrier,efficiency,active,build_year,lifetime,p_nom,p_nom_mod,...,underground,under_construction,tags,geometry,dc,underwater_fraction,energy to power ratio,location,reversed,length_original
Link,,,,,,,,,,,,,,,,,,,,,
DK0 0 methanolisation,DK0 0 H2,EU methanol,,methanolisation,0.8787,True,0,20.0,0.0,0.0,...,NaN,NaN,,,NaN,NaN,NaN,,False,0.0
DK1 0 methanolisation,DK1 0 H2,EU methanol,,methanolisation,0.8787,True,0,20.0,0.0,0.0,...,NaN,NaN,,,NaN,NaN,NaN,,False,0.0
DK0 cluster methanolization,DK0 cluster H2,EU methanol,,methanolisation,0.8787,True,0,20.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN


In [250]:
n.buses

,v_nom,type,x,y,carrier,unit,location,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network,country,substation_lv,substation_off
Bus,,,,,,,,,,,,,,,,
DK0 0,380.0,,9.648393,55.893047,AC,MWh_el,DK0 0,1.0,0.0,inf,Slack,,,DK,1.0,1.0
DK1 0,380.0,,12.303316,55.515974,AC,MWh_el,DK1 0,1.0,0.0,inf,Slack,,,DK,1.0,1.0
EU,1.0,,-5.500000,46.000000,none,,EU,1.0,0.0,inf,PQ,,,,NaN,NaN
co2 atmosphere,1.0,,-5.500000,46.000000,co2,t_co2,EU,1.0,0.0,inf,PQ,,,,NaN,NaN
DK0 0 co2 stored,1.0,,9.648393,55.893047,co2 stored,t_co2,DK0 0,1.0,0.0,inf,PQ,,,DK,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DK1 0 urban decentral heat,1.0,,12.303316,55.515974,urban decentral heat,MWh_th,DK1 0,1.0,0.0,inf,PQ,,,DK,NaN,NaN
DK1 0 urban decentral water tanks,1.0,,12.303316,55.515974,urban decentral water tanks,MWh_th,DK1 0,1.0,0.0,inf,PQ,,,DK,NaN,NaN
DK0 cluster,380.0,,9.648393,55.893047,AC,MWh_el,DK0 0,1.0,0.0,inf,Slack,,,DK,1.0,1.0


In [251]:

n.links["reversed"] = n.links["reversed"].fillna(False).astype(bool)


In [252]:
n.generators
n.generators_t['p_max_pu']


Generator,DK0 0 0 offwind-ac,DK0 0 0 offwind-dc,DK0 0 0 offwind-float,DK0 0 0 onwind,DK0 0 0 solar,DK0 0 0 solar rooftop,DK0 0 0 solar-hsat,DK0 0 1 onwind,DK0 0 1 solar,DK0 0 1 solar rooftop,...,DK1 0 3 onwind,DK1 0 3 solar,DK1 0 3 solar rooftop,DK1 0 4 onwind,DK1 0 4 solar,DK1 0 4 solar rooftop,DK1 0 rural solar thermal collector,DK1 0 urban central solar thermal collector,DK1 0 urban decentral solar thermal collector,DK0 0 cluster 4 onwind
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,0.700927,0.830545,0.770990,0.364236,0.069839,0.069839,0.050814,0.475224,0.058030,0.058030,...,0.961331,0.009358,0.009358,0.979361,0.008680,0.008680,0.000001,0.000001,0.000001,0.696333
2013-01-01 12:00:00,0.759989,0.876464,0.698386,0.420273,0.036241,0.036241,0.025825,0.474261,0.033199,0.033199,...,0.653197,0.003397,0.003397,0.762600,0.004641,0.004641,0.000807,0.000807,0.000807,0.826427
2013-01-02 00:00:00,0.759867,0.871287,0.655037,0.423850,0.070883,0.070883,0.068377,0.516729,0.074624,0.074624,...,0.933312,0.027854,0.027854,0.935317,0.024506,0.024506,0.018721,0.018721,0.018721,0.804040
2013-01-02 12:00:00,0.880320,0.881039,0.773240,0.630831,0.035263,0.035263,0.027083,0.743474,0.033299,0.033299,...,0.987483,0.029269,0.029269,0.979369,0.007522,0.007522,0.008221,0.008221,0.008221,0.942114
2013-01-03 00:00:00,0.885175,0.885500,0.868972,0.953745,0.023163,0.023163,0.034479,0.968734,0.042885,0.042885,...,0.999855,0.020153,0.020153,0.999986,0.013083,0.013083,0.000940,0.000940,0.000940,0.979890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-29 12:00:00,0.848486,0.858358,0.667661,0.441499,0.017080,0.017080,0.015200,0.557151,0.017868,0.017868,...,0.981745,0.004552,0.004552,0.992750,0.006311,0.006311,0.000628,0.000628,0.000628,0.837427
2013-12-30 00:00:00,0.875231,0.885485,0.829320,0.610812,0.039018,0.039018,0.031613,0.701322,0.041004,0.041004,...,0.973625,0.028677,0.028677,0.994667,0.036115,0.036115,0.001827,0.001827,0.001827,0.943882
2013-12-30 12:00:00,0.885500,0.868856,0.835930,0.969331,0.022534,0.022534,0.019495,0.988347,0.025580,0.025580,...,0.991039,0.020721,0.020721,0.978904,0.013289,0.013289,0.000727,0.000727,0.000727,0.999543


In [253]:
n.generators_t['p_max_pu']['DK0 0 cluster 4 onwind'] = n.generators_t['p_max_pu']['DK0 0 4 onwind']


In [254]:
n.links

,bus0,bus1,type,carrier,efficiency,active,build_year,lifetime,p_nom,p_nom_mod,...,underground,under_construction,tags,geometry,dc,underwater_fraction,energy to power ratio,location,reversed,length_original
Link,,,,,,,,,,,,,,,,,,,,,
relation/5487095-400-DC,DK0 0,DK1 0,,DC,0.976096,True,0,inf,600.0,0.0,...,1.0,0.0,relation/5487095,LINESTRING (10.505724427906852 55.365970143543...,1.0,0.560249,NaN,,False,171.53249
DK0 0 co2 sequestered,DK0 0 co2 stored,DK0 0 co2 sequestered,,co2 sequestered,1.000000,True,0,inf,0.0,0.0,...,NaN,NaN,,,NaN,NaN,NaN,,False,0.00000
DK1 0 co2 sequestered,DK1 0 co2 stored,DK1 0 co2 sequestered,,co2 sequestered,1.000000,True,0,inf,0.0,0.0,...,NaN,NaN,,,NaN,NaN,NaN,,False,0.00000
DK0 0 OCGT,DK0 0 gas,DK0 0,,OCGT,0.430000,True,0,25.0,0.0,0.0,...,NaN,NaN,,,NaN,NaN,NaN,,False,0.00000
DK1 0 OCGT,DK1 0 gas,DK1 0,,OCGT,0.430000,True,0,25.0,0.0,0.0,...,NaN,NaN,,,NaN,NaN,NaN,,False,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DK1 0 urban decentral resistive heater,DK1 0 low voltage,DK1 0 urban decentral heat,,urban decentral resistive heater,0.900000,True,0,20.0,0.0,0.0,...,NaN,NaN,,,NaN,NaN,NaN,,False,0.00000
DK1 0 urban decentral water tanks charger,DK1 0 urban decentral heat,DK1 0 urban decentral water tanks,,urban decentral water tanks charger,1.000000,True,0,30.0,0.0,0.0,...,NaN,NaN,,,NaN,NaN,0.15,,False,0.00000
DK1 0 urban decentral water tanks discharger,DK1 0 urban decentral water tanks,DK1 0 urban decentral heat,,urban decentral water tanks discharger,1.000000,True,0,30.0,0.0,0.0,...,NaN,NaN,,,NaN,NaN,NaN,,False,0.00000


In [255]:
n.export_to_netcdf(fn)


INFO:pypsa.network.io:Exported network 'Unnamed Network'saved to 'resources/DK_test/networks/base_s_2__12h_2050.nc contains: carriers, global_constraints, stores, generators, loads, links, buses


<xarray.Dataset> Size: 509kB
Dimensions:                               (snapshots: 730,
                                           investment_periods: 0,
                                           carriers_i: 118,
                                           global_constraints_i: 3,
                                           stores_i: 28,
                                           stores_t_e_min_pu_i: 2,
                                           ...
                                           generators_i: 60,
                                           generators_t_p_max_pu_i: 45,
                                           loads_i: 37, loads_t_p_set_i: 10,
                                           links_i: 137,
                                           links_t_efficiency_i: 8,
                                           links_t_p_max_pu_i: 4, buses_i: 61)
Coordinates: (12/15)
  * snapshots                             (snapshots) int64 6kB 0 1 ... 728 729
  * investment_periods                    (investment_periods) object 0B 
  * carriers_i                            (carriers_i) object 944B 'AC' ... '...
  * global_constraints_i                  (global_constraints_i) object 24B '...
  * stores_i                              (stores_i) object 224B 'co2 atmosph...
  * stores_t_e_min_pu_i                   (stores_t_e_min_pu_i) object 16B 'D...
    ...                                    ...
  * loads_i                               (loads_i) object 296B 'DK0 0' ... '...
  * loads_t_p_set_i                       (loads_t_p_set_i) object 80B 'DK0 0...
  * links_i                               (links_i) object 1kB 'relation/5487...
  * links_t_efficiency_i                  (links_t_efficiency_i) object 64B '...
  * links_t_p_max_pu_i                    (links_t_p_max_pu_i) object 32B 'DK...
  * buses_i                               (buses_i) object 488B 'DK0 0' ... '...
Data variables: (12/90)
    snapshots_snapshot                    (snapshots) datetime64[ns] 6kB 2013...
    snapshots_objective                   (snapshots) float64 6kB 12.0 ... 12.0
    snapshots_stores                      (snapshots) float64 6kB 12.0 ... 12.0
    snapshots_generators                  (snapshots) float64 6kB 12.0 ... 12.0
    investment_periods_objective          (investment_periods) float64 0B 
    investment_periods_years              (investment_periods) float64 0B 
    ...                                    ...
    buses_unit                            (buses_i) object 488B 'MWh_el' ... ...
    buses_location                        (buses_i) object 488B 'DK0 0' ... '...
    buses_control                         (buses_i) object 488B 'Slack' ... 'PQ'
    buses_country                         (buses_i) object 488B 'DK' ... 'DK'
    buses_substation_lv                   (buses_i) float64 488B 1.0 1.0 ... nan
    buses_substation_off                  (buses_i) float64 488B 1.0 1.0 ... nan
Attributes:
    network__multi_invest:  0
    network_name:           Unnamed Network
    network_pypsa_version:  0.35.2
    network_srid:           4326
    crs:                    {"_crs": "GEOGCRS[\"WGS 84\",ENSEMBLE[\"World Geo...
    meta:                   {"version": "v2025.07.0", "tutorial": false, "log...